In [1]:
import numpy as np
from pathlib import Path
import shutil
import warnings

from eddymotion.data import pet
from eddymotion.model import PETModel
from eddymotion.viz import plot_dwi
import nibabel as nib
from eddymotion.data.splitting import lovo_split

In [2]:
base_dir = Path("/Users/martinnorgaard/Documents/BIDS/")
bids_dir = base_dir / "ds005138-download"
derivatives_dir = base_dir / "eddymotion"

pet_file = bids_dir / "sub-01" / "ses-baseline" / "pet" / "sub-01_ses-baseline_pet.nii.gz"
json_file = bids_dir / "sub-01" / "ses-baseline" / "pet" / "sub-01_ses-baseline_pet.json"

pet_dataset = pet.PET.load(pet_file, json_file)

In [7]:
data_train, data_test = pet_dataset.lofo_split(15)

In [8]:
model = PETModel(timepoints=data_train[1], xlim = 7000)

In [9]:
model.fit(data_train[0])

In [6]:
predicted = model.predict(data_test[1])

In [7]:
# before
nifti_img_before = nib.Nifti1Image(predicted, pet_dataset.affine)
output_path_before = 'before_mc.nii'
nifti_img_before.to_filename(output_path_before)

# after
nifti_img_after = nib.Nifti1Image(data_test[0], pet_dataset.affine)
output_path_after = 'after_mc.nii'
nifti_img_after.to_filename(output_path_after)

In [5]:
from niworkflows.viz.notebook import display
import os
from IPython.display import SVG

moving_image = output_path_after
fixed_image = output_path_before
obj = display(fixed_image, moving_image, fixed_label='PET_before', moving_label='PET')

NameError: name 'output_path_after' is not defined

In [3]:
from eddymotion.estimator import PETMotionEstimator

In [4]:
estimator = PETMotionEstimator.estimate(pet_dataset, omp_nthreads=4, n_jobs=4)

Estimating PET motion: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:43<00:00,  2.07s/it]


In [6]:
estimator

[array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 array([[1., 0., 0., 0.],
        [0., 1., 0.,

In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

# For IPython or Python script, you might use
import matplotlib
matplotlib.use('TkAgg')

def extract_motion_parameters(affine):
    # Translation is straightforward
    tx, ty, tz = affine[0, 3], affine[1, 3], affine[2, 3]
    
    # For small rotations, these approximations are usually sufficient
    rx = np.arctan2(affine[2, 1], affine[2, 2])
    ry = np.arctan2(-affine[2, 0], np.sqrt(affine[2, 1]**2 + affine[2, 2]**2))
    rz = np.arctan2(affine[1, 0], affine[0, 0])
    
    return tx, ty, tz, np.degrees(rx), np.degrees(ry), np.degrees(rz)

In [11]:
# Assuming 'affines' is the list of affine matrices you have
motion_parameters = []
affines = estimator

for idx, affine in enumerate(affines):
    tx, ty, tz, rx, ry, rz = extract_motion_parameters(affine)
    motion_parameters.append({
        "Frame": idx,
        "Translation X": tx,
        "Translation Y": ty,
        "Translation Z": tz,
        "Rotation X (deg)": rx,
        "Rotation Y (deg)": ry,
        "Rotation Z (deg)": rz
    })

df_motion = pd.DataFrame(motion_parameters)

In [ ]:
# Set up the matplotlib figure
plt.figure(figsize=(18, 10))

# Translation plots
plt.subplot(2, 3, 1)
sns.lineplot(x='Frame', y='Translation X', data=df_motion)
plt.title('Translation X over Frames')

plt.subplot(2, 3, 2)
sns.lineplot(x='Frame', y='Translation Y', data=df_motion)
plt.title('Translation Y over Frames')

plt.subplot(2, 3, 3)
sns.lineplot(x='Frame', y='Translation Z', data=df_motion)
plt.title('Translation Z over Frames')

# Rotation plots
plt.subplot(2, 3, 4)
sns.lineplot(x='Frame', y='Rotation X (deg)', data=df_motion)
plt.title('Rotation X over Frames')

plt.subplot(2, 3, 5)
sns.lineplot(x='Frame', y='Rotation Y (deg)', data=df_motion)
plt.title('Rotation Y over Frames')

plt.subplot(2, 3, 6)
sns.lineplot(x='Frame', y='Rotation Z (deg)', data=df_motion)
plt.title('Rotation Z over Frames')

plt.tight_layout()
plt.show()